In [ ]:
import sys

import os
import argparse
import tensorrt as trt

EXPLICIT_BATCH = 1 << (int)(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)

onnx_file_path = "/home/yuzn/文档/deploy0222/quick_sentence_transformers/onnx_models/Multilingual_MiniLM_L12.onnx"
engine_file_path = "/home/yuzn/文档/deploy0222/quick_sentence_transformers/onnx_models/test_v1.plan"
print('get start')
TRT_LOGGER = trt.Logger()
with trt.Builder(TRT_LOGGER) as builder, builder.create_network(EXPLICIT_BATCH) as network, trt.OnnxParser(network, TRT_LOGGER) as parser:
    config = builder.create_builder_config()
    #builder.max_workspace_size =( 1 << 30 ) * 2
    config.max_workspace_size =( 1 << 20 ) * 3 * 1024 # 3GB，可以根据需求改的更大
    builder.max_batch_size = 128
    config.set_flag(trt.BuilderFlag.FP16)
    #builder.fp16_mode = True
    # Parse model file
if not os.path.exists(onnx_file_path):
    print('ONNX file {} not found, please run yolov3_to_onnx.py first to generate it.'.format(onnx_file_path))
    exit(0)
print('Loading ONNX file from path {}...'.format(onnx_file_path))
with open(onnx_file_path, 'rb') as model:
    print('Beginning ONNX file parsing')
    if not parser.parse(model.read()):
        print ('ERROR: Failed to parse the ONNX file.')
        for error in range(parser.num_errors):
            print (parser.get_error(error))
print(f"raw shape of {network.get_input(0).name} is: ", network.get_input(0).shape)
profile = builder.create_optimization_profile()
for temp_name in ['input_ids', 'attention_mask','token_type_ids']:
    profile.set_shape(
        input=temp_name,
        min=(1,1),
        opt=(1,64),
        max=(64,128),
    )
config.add_optimization_profile(profile)
print('Completed parsing of ONNX file')
print('Building an engine from file {}; this may take a while...'.format(onnx_file_path))
engine = builder.build_engine(network,config)
print("Completed creating Engine")
with open(engine_file_path, "wb") as f:
    f.write(engine.serialize())